<a href="https://colab.research.google.com/github/Ech-Charay/Corona-FAQs-chatbot/blob/master/corona_FAQs_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Installing prerequisites:**

In [6]:
!pip install flask-ngrok

In [7]:
!wget "https://download.pytorch.org/models/tutorials/4000_checkpoint.tar" 

--2020-07-13 23:42:21--  https://download.pytorch.org/models/tutorials/4000_checkpoint.tar
Resolving download.pytorch.org (download.pytorch.org)... 13.35.34.50, 13.35.34.52, 13.35.34.129, ...
Connecting to download.pytorch.org (download.pytorch.org)|13.35.34.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 257811088 (246M) [application/x-tar]
Saving to: ‘4000_checkpoint.tar.2’

4000_checkpoint.tar 100%[===================>] 245.87M  26.0MB/s    in 11s     

2020-07-13 23:42:33 (23.1 MB/s) - ‘4000_checkpoint.tar.2’ saved [257811088/257811088]



In [8]:
!pip3 install SpeechRecognition

In [9]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install pyaudio

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libportaudio2 is already the newest version (19.6.0-1).
libportaudiocpp0 is already the newest version (19.6.0-1).
portaudio19-dev is already the newest version (19.6.0-1).
libasound2-dev is already the newest version (1.1.3-5ubuntu0.5).
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [10]:
!pip install ffmpeg-python

In [11]:
!pip install gtts

In [12]:
!mkdir -p /content/records/in /content/records/out

# **First part of the chatbot: cosine-similarity based Retrieval chatbot**

In [13]:
# Preprocess raw text Excel file into CSV using spaCy to tokenize sentences

import pandas as pd
import unicodedata
import spacy


def preprocess(entry):
    '''
    Normalize string argument entry, remove new line characters,
    add space after '?' chars, and return entry with trailing space.
    '''
    entry = entry.replace('\n', ' ')
    entry = unicodedata.normalize("NFKD", entry)
    entry = entry.replace('?', '? ')
    return entry + ' '


def sentence_tokenize(entry):
    '''
    Tokenize string argument entry into sentences using spaCy,
    then join sentences with double new lines and return as string.
    '''
    doc = nlp(entry)
    sentences = list(doc.sents)
    sentences = '\n\n'.join([(s.text) for s in sentences])
    return sentences

    
def paragraph_tokenize(entry):
    '''
    Tokenize string argument entry into paragraphs,
    then join sentences with double new lines and return as string.
    '''

    bloc = []
    paragraphs = []
    for s in entry.split('\n'):
      if len(s) == 0:
        continue
      elif (s[0] == '-' or s[0] == '*' or s[0] == '+' or (s[0] >= '0' and s[0] <= '9')) :
        bloc.append(s)
      elif len(bloc) != 0:
        paragraphs.append('\n'.join(bloc))
        paragraphs.append(s)
        bloc = []
      else :
        paragraphs.append(s)
    if len(bloc) != 0:
      paragraphs.append('\n'.join(bloc))
    return '\n\n'.join(paragraphs)

In [14]:
spacy_model = 'en_core_web_sm'
print('Loading spaCy model', spacy_model, '...')
nlp = spacy.load(spacy_model)
print('-Done.')
faq = pd.read_excel("/content/drive/My Drive/MyFirstChatBot/data/Q&A_COVID-19.xlsx")
#faq = raw.drop(labels=('Active'), axis=1).dropna()
#faq.question = faq.question.apply(preprocess)
#faq.question = faq.question.apply(sentence_tokenize)
#faq.answer = faq.answer.apply(preprocess)
#faq.answer = faq.answer.apply(sentence_tokenize)
faq.answer = faq.answer.apply(paragraph_tokenize)
faq.to_csv('/content/drive/My Drive/MyFirstChatBot/data/faq-text-preprocessed.csv', index=False)

Loading spaCy model en_core_web_sm ...
-Done.


In [15]:
import string
from sklearn.feature_extraction import stop_words
from nltk.stem import WordNetLemmatizer
import nltk

class Processing:
  def __init__(self):
    print("init processing")
    nltk.download('wordnet')

  def lem(self, words):
      """Returns list of lemmas from argument list of words."""
      wordnet_lemmatizer = WordNetLemmatizer()
      lem_sentence = []
      for word in words:
          lem_sentence.append(wordnet_lemmatizer.lemmatize(word))
      return lem_sentence


  def text_process(self, mess, lemmas=True):
      """
      Returns list of tokenized lemmas in argument string mess, with stopwords,
      punctuation removed.
      """
      clean = [char if char not in string.punctuation else ' ' for char in mess]
      clean = ''.join(clean)
      clean = [word.lower() for word in clean.split() if word.lower()
              not in stop_words.ENGLISH_STOP_WORDS]
      
      if lemmas:
          clean = self.lem(clean)
          
      return clean

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


# **Second part of the chatbot: Generative smart chatbot**

In [16]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
import torch.nn as nn
import torch.nn.functional as F
import re
import os
import unicodedata
import numpy as np

device = torch.device("cpu") # TODO mn l2fdal khlliha cpu bax maykerjox lik machakil mn jenb khossosan trained model mtraine 3la cpu


MAX_LENGTH = 10  # Maximum sentence length

# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

In [17]:
class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True
        keep_words = []
        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))
        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens
        for word in keep_words:
            self.addWord(word)


# Lowercase and remove non-letter characters
def normalizeString(s):
    s = s.lower()
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s


# Takes string sentence, returns sentence of word indexes
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


In [18]:
class EncoderRNN(nn.Module):

    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # type: (Tensor, Tensor, Optional[Tensor]) -> Tuple[Tensor, Tensor]
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

In [19]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [20]:

class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

In [21]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder, decoder_n_layers):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self._device = device
        self._SOS_token = SOS_token
        self._decoder_n_layers = decoder_n_layers

    __constants__ = ['_device', '_SOS_token', '_decoder_n_layers']

    def forward(self, input_seq : torch.Tensor, input_length : torch.Tensor, max_length : int):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:self._decoder_n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=self._device, dtype=torch.long) * self._SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=self._device, dtype=torch.long)
        all_scores = torch.zeros([0], device=self._device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

In [22]:
save_dir = os.path.join("data", "save")
corpus_name = "cornell movie-dialogs corpus"

# Configure models
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# If you're loading your own model
# Set checkpoint to load from
checkpoint_iter = 4000
# loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                             '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                             '{}_checkpoint.tar'.format(checkpoint_iter))

# If you're loading the hosted model
loadFilename = '/content/4000_checkpoint.tar'

# Load model
# Force CPU device options (to match tensors in this tutorial)
checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
encoder_sd = checkpoint['en']
decoder_sd = checkpoint['de']
encoder_optimizer_sd = checkpoint['en_opt']
decoder_optimizer_sd = checkpoint['de_opt']
embedding_sd = checkpoint['embedding']
voc = Voc(corpus_name)
voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
# Load trained model params
encoder.load_state_dict(encoder_sd)
decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
# Set dropout layers to eval mode
encoder.eval()
decoder.eval()
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [23]:
def evaluate(searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words



from gtts import gTTS #Import Google Text to Speech
from IPython.display import Audio  



# Evaluate inputs from user input (stdin)
def evaluateInput(searcher, voc,input_sentence):
    #input_sentence = ''
    while(1):
        try:
            # Get input sentence
            #input_sentence = input('> ')
            # Check if it is quit case
            #if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = evaluate(searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            engine = gTTS(''.join(output_words))  
            engine.save('a1.wav') 
            Audio('a1.wav', autoplay=True)  
            print('Bot:', ' '.join(output_words))
        except KeyError:
            print("Error: Encountered unknown word.")



# Evaluate inputs from user input (stdin)
def evaluateInput2(searcher, voc):
  input_sentence = ''
  try:
    # Get input sentence
    input_sentence = input('> ')
    # Check if it is quit case
    # Normalize sentence
    input_sentence = normalizeString(input_sentence)
    # Evaluate sentence
    output_words = evaluate(searcher, voc, input_sentence)
    # Format and print response sentence
    output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
    print('Bot:', ' '.join(output_words))
    bot_answer=' '.join(output_words)
    engine = gTTS(''+bot_answer)  
    engine.save('a1.wav') 
  except KeyError:
    print("Error: Encountered unknown word.")





# Normalize input sentence and call evaluate()
def evaluateExample(sentence, searcher, voc):
    print("> " + sentence)
    # Normalize sentence
    input_sentence = normalizeString(sentence)
    # Evaluate sentence
    output_words = evaluate(searcher, voc, input_sentence)
    output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
    print('Bot:', ' '.join(output_words))

In [24]:
# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder,decoder_n_layers)
#evaluateInput2(searcher,voc) TODO t9der tkhdem bhad l fonction fiha while ms kadir dal lprblm li dwit lik 3lih

# **Bot Server:**

In [30]:
import os
import numpy as np
import pandas as pd

import librosa
from datetime import datetime
import random 

#import processing

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity

from flask import jsonify, request
from werkzeug.utils import secure_filename


class BotServer:
    def __init__(self, file_path):
        """
        Initialize corpus, bag-of-words, and TFIDF from CSV file at argument
        file_path.
        """
        processing = Processing()
        # Read in FAQ data
        self.faq = pd.read_csv(file_path, keep_default_na=False)
        self.corpus = self.faq.question + ' ' + self.faq.answer

        # Create BOW tranformer based on faq.question + faq.answer
        self.bow_transformer = CountVectorizer(analyzer=processing.text_process).fit(self.faq.question)
        # Tranform faq.question itself into BOW
        self.corpus_bow = self.bow_transformer.transform(self.faq.question)

        # Create TFIDF transformer based on faq.question's BOW
        self.tfidf_transformer = TfidfTransformer().fit(self.corpus_bow)
        # Transform faq.question's BOW into TFIDF
        self.corpus_tfidf = self.tfidf_transformer.transform(self.corpus_bow)

        # Set upload folder and output records folder
        self.UPLOAD_FOLDER = '/content/records/in'
        self.REC_RES_FOLDER = '/content/records/out'

        # Set allowed extensions
        self.ALLOWED_EXTENSIONS = {'wav'}

    def tfidf_similarity(self, query):
        """
        Returns (index, similarity value) of string argument query's most similar
        match in FAQ, determined by cosine similarity.
        """
        # Transform test question into BOW using BOW transformer
        query_bow = self.bow_transformer.transform([query])
        # Transform test question's BOW into TFIDF
        query_tfidf = self.tfidf_transformer.transform(query_bow)

        # Calculate cosine similarity and return maximum value with accompanying index
        similarities = np.transpose(cosine_similarity(query_tfidf, self.corpus_tfidf))
        max_similarity = similarities.max()
        max_index = np.argmax(similarities)

        return max_index, max_similarity

    def match_query(self, query):
        """
        Prints most similar match in FAQ to user query.
        """
        index, similarity = self.tfidf_similarity(query)

        if similarity > 0.5 :
          response = self.faq.answer.iloc[index]
          print(similarity)
        else :
          query = normalizeString(query)
          output_words = evaluate(searcher, voc, query)
          output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
          response = ' '.join(output_words)
       
        return response

    def allowed_file(self, filename):
      return '.' in filename and filename.rsplit('.', 1)[1].lower() in self.ALLOWED_EXTENSIONS

    def bot_dialog(self, request):
        """
        Given the argument POST request, parse it according to json or form data,
        and return a json response or html template based on sklearn matching
        within the FAQ.
        """
        
        print('****************')
        # Handle webhook request
        #.get_json(force=True)
        req = request.form
        msg_type = req.get('type')
        if msg_type == "Text":
          message = req.get('message')
          response_text = self.match_query(message)
          # Return json file as webhook response 
          messages = [
                      {
                          "type": "Text",
                          "message": msg,
                          "fromBot": True
                      }
                      for msg in response_text.split("\n\n")
                      ]
        elif msg_type == "Audio":
          record = request.files['record']
          if record and self.allowed_file(record.filename):
            filename = secure_filename(record.filename)
            record.save(os.path.join(self.UPLOAD_FOLDER, filename))
          list_records = []
          try:
            r = spechrec.Recognizer()
            with spechrec.AudioFile(os.path.join(self.UPLOAD_FOLDER, filename)) as source:
              # listen for the data (load audio to memory)
              audio_data = r.record(source)
              # recognize (convert from speech to text)
              input_sentence = r.recognize_google(audio_data)
            response_text = self.match_query(searcher, voc, input_sentence)
            for msg in response_text.split("\n\n"):
              now = datetime.now()
              respfilename = now.strftime("%d-%m-%Y-%H:%M:%S") + ".wav"
              engine = gTTS('' + response_text)
              engine.save(os.path.join(self.REC_RES_FOLDER, respfilename))
              list_records.append(respfilename)
          except:
            erreur = random.choice(["Sorry, i did not understand you ,Please change the way you say it",
                          "please be a little simple in your discussion i m not a human",
                          "Sorry, get in mind  that you are talking only with a computer "])
            print(""+erreur)
            now = datetime.now()
            respfilename = now.strftime("%d-%m-%Y-%H:%M:%S") + ".wav"
            engine = gTTS(''+erreur)
            engine.save(os.path.join(self.REC_RES_FOLDER, respfilename))
            list_records.append(respfilename)
          duration = round(librosa.get_duration(filename=os.path.join(self.REC_RES_FOLDER, respfilename)))
          # Return json file as webhook response 
          messages = [
                      {
                          "type": "Audio",
                          "path": "http://b1ec39a3df14.ngrok.io/records/"+respfilename,
                          "isLocal": False,
                          "duration": duration,
                          "fromBot": True
                      }
                      for filename in list_records
                      ]
        return jsonify({"messages": messages})

# **Main APP:**

In [26]:
bot = BotServer('/content/drive/My Drive/MyFirstChatBot/data/faq-text-preprocessed.csv')
while(1):
  try:
    # Get input sentence
    input_sentence = input('> ')
    # Check if it is quit case
    if input_sentence == 'q' or input_sentence == 'quit': break
    # Evaluate sentence
    output_words = bot.match_query(input_sentence)
    print('Bot:', ''.join(output_words))
  except KeyError:
    print("Error: Encountered unknown word.")

init processing
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
> q


In [31]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify, json, send_from_directory

app = Flask(__name__, static_url_path='')
run_with_ngrok(app)   #starts ngrok when the app is run

@app.route('/')
def index():
    """
    This route at the home page renders the index.html template with blank
    variables.
    """
    
    return jsonify(
        messages = [
            {
                "message": "Hello!",
                "fromBot": True
            },
            {
                "message": "How are you?",
                "fromBot": True
            },
            {
                "message": "You can ask me what ever you want about coronavirus.",
                "fromBot": True
            }
        ]
    )

@app.route('/dialog', methods=['POST'])
def dialog():
    """
    This route uses the POST method to take user request and return the result
    of passing it to the BotServer similarity matching function.
    """
    return bot.bot_dialog(request)

@app.route('/records/<path:filename>', methods=['GET', 'POST'])
def download(filename):
    return send_from_directory(directory='/content/records/out', filename=filename)


if __name__ == '__main__':
    """
    Default Python entrypoint creates Botserver object and runs Flask app.
    """
    bot = BotServer('/content/drive/My Drive/MyFirstChatBot/data/faq-text-preprocessed.csv')

    app.run()

init processing
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e2bf0db4d9dd.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
****************
Sorry, get in mind  that you are talking only with a computer 


127.0.0.1 - - [13/Jul/2020 23:57:40] "POST /dialog HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2020 23:58:40] "GET /records/13-07-2020-23:57:39.wav HTTP/1.1" 200 -
